In [1]:
import pandas as pd
import pandas_ta as ta
from math import *



def stoch_rsi(df,lookback=14):
    '''
    calculate and add the stock_rsi to the input dataframe
    '''

    #use pandas_ta to calculate the rsi
    rsi = df.ta.rsi(length = lookback)

    #convert rsi to stoch rsi, equation: (current rsi - min rsi) / (max rsi - min rsi)
    df['stoch_rsi'] = (rsi.iloc[-1] - rsi.tail(lookback).min()) / (rsi.tail(lookback).max() - rsi.tail(lookback).min())



def nadaraya(df,bandwidth=8,source='close'):
    '''
    calculate the nadarya indicator and return the most recent buy/sell signals
    '''
    src = df[source]

    out = []
    
    for i,v1 in src.iteritems():
        tsum = 0
        sumw = 0
        for j,v2 in src.iteritems():
            w = exp(-(pow(i-j,2)/(bandwidth*bandwidth*2)))
            tsum += v2*w
            sumw += w
        out.append(tsum/sumw)
        
    df['nadarya'] = out
    d = df['nadarya'].rolling(window=2).apply(lambda x: x.iloc[1] - x.iloc[0])
    df['nadarya_buy'] = (d > 0) & (d.shift(1) < 0)
    df['nadarya_sell'] = (d < 0) & (d.shift(1) > 0)

    return df['nadarya_buy'].iloc[-1], df['nadarya_sell'].iloc[-1] #returns buy signal, sell signal




def atr(df):
    '''
    calculate and add the true range to the input dataframe. Returns the most recent true range
    '''

    #use pandas_ta to calculate the true range
    df['True_Range'] = df.ta.atr()

    return df['True_Range'].iloc[-1]



def wma(data,length):
    wma = ta.wma(data,length=length)
    return wma

def ema(data,length):
    ema = ta.ema(data,length=length)
    return ema



def hull(df,mode='hma',length=60,lengthMult=3):
    '''mode: choice of hma, thma, ehma
    df: a dataframe of candles data
    length: 180-200 for floating S/R , 55 for swing entry
    lengthMult: Used to view higher timeframes with straight band
    '''


    #CALCULATE DIFFERENT MOVING AVERAGES BASED ON MODE
    def hma(df,length):
        hma = wma( (2 * wma(df['close'],length/2)) - wma(df['close'],length), round(sqrt(length)) )
        return hma

    def ehma(df,length):
        ehma = ema( (2 * ema(df['close'],length/2)) - ema(df['close'],length), round(sqrt(length)) )
        return ehma

    def thma(df, length):
        thma = wma( (3 * wma(df['close'], length/3)) - wma(df['close'], length/2) - wma(df['close'],length), length)
        return thma

    def Mode(mode,df,length):
        if mode == 'hma':
            return hma(df,length)
        elif mode == 'ehma':
            return ehma(df,length)
        else:
            return thma(df,length/2)


    df['hull'] = Mode(mode,df,int(length*lengthMult))

    df['hull_green'] = df['hull'] > df['hull'].shift(2)
    df['closeover'] = df['close'] > df['hull']

    df['hull_red'] = df['hull'] < df['hull'].shift(2)
    df['closeunder'] = df['close'] < df['hull']

    df['trending_up'] = (df['hull'] > df['hull'].shift(2)) & (df['hull'].shift(1) < df['hull'].shift(3))
    df['trending_down'] = (df['hull'] < df['hull'].shift(2)) & (df['hull'].shift(1) > df['hull'].shift(3))

    return df.iloc[:, 6:]








'''
 ==================================================

 VOLUME OSCILLATOR - must be > 0% to enter

 ==================================================
'''

def calc_vol_osc(df,shortLen=5,longLen=10):
    '''df: a dataframe of candles data
    shortLen: length of long line
    longLen: length of short line'''

    df['cumVol'] = df['volume'].cumsum()

    short = ema(df['volume'],shortLen)
    long = ema(df['volume'],longLen)
    osc = 100 * ((short-long) / long)

    df['VolOsc'] = osc
    df['VolOsc_over_0'] = df['VolOsc'] > 0
    df['VolOsc_under_0'] = df['VolOsc'] < 0

    return df.iloc[:, 6:]










'''
 ==================================================

 QQE - must be > 0% to enter

 ==================================================
'''




def qqe(df,rsi_period=6,rsi_smoothing=5,qqe=3,boll_length=50,boll_mult=.35,rsi_period2=6,rsi_smoothing2=5,qqe2=1.61,threshhold2=3):
    '''rsi_period: length of rsi
    rsi_smoothing: RSI Smoothing Factor
    qqe: Fast QQE Factor
    boll_length: length of bbands
    boll_mult: mulitplier value for bbands
    rsi_period2: RSI Length
    rsi_smoothing2: RSI Smoothing
    qqe2: Fast QQE2 Factor
    threshhold2: Thresh-hold'''

    wilders_period = rsi_period*2-1

    df['rsi'] = df.ta.rsi(length=rsi_period)
    df['rsi_ema'] = df.ta.ema(close=df['rsi'],length=rsi_smoothing)
    df['atr_rsi'] = abs(df['rsi_ema'].shift(1) - df['rsi_ema'])
    df['ema_atr_rsi'] = df.ta.ema(close=df['atr_rsi'],length=wilders_period)

    df['dar'] = df.ta.ema(close=df['ema_atr_rsi'],length=wilders_period)*qqe



    df['new_shortband'] = df['rsi_ema'] + df['dar']
    df['new_longband'] = df['rsi_ema'] - df['dar']


    longband = [0]
    shortband = [0]
    for index,row in df.iterrows():
        l = max(longband[-1],row['new_longband']) if (df['rsi_ema'].iloc[index-1] > longband[-1]) and (row['rsi_ema'] > longband[-1]) else row['new_longband']
        longband.append(l)

        s = max(shortband[-1],row['new_shortband']) if (df['rsi_ema'].iloc[index-1] < shortband[-1]) and (row['rsi_ema'] < shortband[-1]) else row['new_shortband']
        shortband.append(s)

    df['longband'] = longband[1:]   
    df['shortband'] = shortband[1:]


    #set cross_1 column to True if rsi_ema and longband crossed
    df['cross_1'] = (((df['rsi_ema'] > df['longband']) & (df['rsi_ema'].shift(1) < df['longband'].shift(1))) | ((df['rsi_ema'] < df['longband']) & (df['rsi_ema'].shift(1) > df['longband'].shift(1))))
    
    #set trend column to 1 if rsi_ema and shortband crossed, -1 if they didnt but cross_1 column is True, and otherwise 1
    df.loc[((df['rsi_ema'] > df['shortband']) & (df['rsi_ema'].shift(1) < df['shortband'].shift(1))) | ((df['rsi_ema'] < df['shortband']) & (df['rsi_ema'].shift(1) > df['shortband'].shift(1))), 'trend'] = 1
    df.loc[((df['trend'].isnull()) & (df['cross_1'] == True)),'trend'] = -1
    df['trend'] = df['trend'].fillna(1)
    
    #set fast_atr_rsi_tl column to longband if the trend is 1 else shortband
    df.loc[df['trend'] == 1,'fast_atr_rsi_tl'] = df['longband']
    df['fast_atr_rsi_tl'] = df['fast_atr_rsi_tl'].fillna(df['shortband'])
    
    
    df['basis'] = df.ta.sma(close=df['fast_atr_rsi_tl']-50,length=boll_length)
    df['dev'] = boll_mult * df.ta.stdev(close=df['fast_atr_rsi_tl']-50,length=boll_length)
    df['upper'] = df['basis'] + df['dev']
    df['lower'] = df['basis'] - df['dev']


    #set the colors
    df.loc[df['rsi_ema']-50 > df['upper'],'color'] = 'blue'
    df.loc[df['rsi_ema']-50 < df['lower'],'color'] = 'red'
    df['color'] = df['color'].fillna('grey')


    qqe_long = [0]
    qqe_short = [0]
    for index,row in df.iterrows():
        l = qqe_long[-1] + 1 if row['rsi_ema'] >= 50 else 0
        qqe_long.append(l)
        s = qqe_short[-1] + 1 if row['rsi_ema'] < 50 else 0
        qqe_short.append(s)
    df['qqe_long'] = qqe_long[1:]
    df['qqe_short'] = qqe_short[1:]



    ############################################################
    wilders_period2 = rsi_period2*2-1

    df['rsi2'] = df.ta.rsi(length=rsi_period2)
    df['rsi_ema2'] = df.ta.ema(close=df['rsi2'],length=rsi_smoothing2)
    df['atr_rsi2'] = abs(df['rsi_ema2'].shift(1) - df['rsi_ema2'])
    df['ema_atr_rsi2'] = df.ta.ema(close=df['atr_rsi2'],length=wilders_period2)

    df['dar2'] = df.ta.ema(close=df['ema_atr_rsi2'],length=wilders_period2)*qqe2


    df['new_shortband2'] = df['rsi_ema2'] + df['dar2']
    df['new_longband2'] = df['rsi_ema2'] - df['dar2']

    longband2 = [0]
    shortband2 = [0]
    for index,row in df.iterrows():
        l = max(longband2[-1],row['new_longband2']) if (df['rsi_ema2'].iloc[index-1] > longband2[-1]) and (row['rsi_ema2'] > longband2[-1]) else row['new_longband2']
        longband2.append(l)

        s = max(shortband2[-1],row['new_shortband2']) if (df['rsi_ema2'].iloc[index-1] < shortband2[-1]) and (row['rsi_ema2'] < shortband2[-1]) else row['new_shortband2']
        shortband2.append(s)

    df['longband2'] = longband2[1:]   
    df['shortband2'] = shortband2[1:]

    #set cross_2 column to True if rsi_ema and longband crossed
    df['cross_2'] = (((df['rsi_ema2'] > df['longband2']) & (df['rsi_ema2'].shift(1) < df['longband2'].shift(1))) | ((df['rsi_ema2'] < df['longband2']) & (df['rsi_ema2'].shift(1) > df['longband2'].shift(1))))
    
    #set trend2 column to 1 if rsi_ema and shortband crossed, -1 if they didnt but cross_1 column is True, and otherwise 1
    df.loc[((df['rsi_ema2'] > df['shortband2']) & (df['rsi_ema2'].shift(1) < df['shortband2'].shift(1))) | ((df['rsi_ema2'] < df['shortband2']) & (df['rsi_ema2'].shift(1) > df['shortband2'].shift(1))), 'trend2'] = 1
    df.loc[((df['trend2'].isnull()) & (df['cross_2'] == True)),'trend2'] = -1
    df['trend2'] = df['trend2'].fillna(1)
    
    #set fast_atr_rsi_tl2 column to longband if the trend is 1 else shortband
    df.loc[df['trend2'] == 1,'fast_atr_rsi_tl2'] = df['longband2']
    df['fast_atr_rsi_tl2'] = df['fast_atr_rsi_tl2'].fillna(df['shortband2'])
    
    
    qqe_long2 = [0]
    qqe_short2 = [0]
    for index,row in df.iterrows():
        l = qqe_long2[-1] + 1 if row['rsi_ema2'] >= 50 else 0
        qqe_long2.append(l)
        s = qqe_short2[-1] + 1 if row['rsi_ema2'] < 50 else 0
        qqe_short2.append(s)
    df['qqe_long2'] = qqe_long2[1:]
    df['qqe_short2'] = qqe_short2[1:]


    #set the colors
    df.loc[((df['rsi_ema2']-50 > threshhold2) | (df['rsi_ema2']-50 < 0 - threshhold2)),'color2'] = 'silver'


    df['greenbar1'] = df['rsi_ema2'] - 50 > threshhold2
    df['greenbar2'] = df['rsi_ema'] - 50 > df['upper']

    df['redbar1'] = df['rsi_ema2'] - 50 < 0 - threshhold2
    df['redbar2'] = df['rsi_ema'] - 50 < df['lower']

    return df.iloc[:, 6:]

ModuleNotFoundError: No module named 'pandas_ta'